# Thu thập dữ liệu cổ phiếu Amazon từ trang web
https://finance.yahoo.com/quote/AMZN/history?period1=1495324800&period2=1653091200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import pandas as pd

    Tạo ra một webdriver để truy cập vào web với các options: web ẩn danh và window size là 1920x1080

In [5]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")

In [6]:
driver = webdriver.Chrome(chrome_options=chrome_options)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4452\1680646079.py:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


    Đưa link web vào driver để lấy được dữ liệu

In [7]:
url = "https://finance.yahoo.com/quote/AMZN/history?period1=1432166400&period2=1653091200&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
driver.get(url)

    Dùng BeautifulSoup để tiến hành phân tích cú pháp web

In [8]:
HTMLPage = BeautifulSoup(driver.page_source, 'html.parser')

    Lấy ra thành phần table với class là 'W(100%) M(0)' lưu vào trong biến Table

In [9]:
Table = HTMLPage.find('table', class_='W(100%) M(0)')

    Lấy ra các hàng có thẻ tr với class là 'BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)' lưu vào trong Rows

In [10]:
Rows = Table.find_all('tr', class_='BdT Bdc($seperatorColor) Ta(end) Fz(s) Whs(nw)')

    Lấy dữ liệu của các cột trong từng thẻ tr

In [11]:
# Tạo ra một mảng trống để chứa toàn bộ dữ liệu
extracted_data = []

# Lặp từng hàng trong Rows
for i in range(0, len(Rows)):
 try:
  # Tạo ra một dictionary trống chứa thông tin các cột của mỗi hàng
  RowDict = {}

  # Lấy ra các thẻ td chứa dữ liệu các cột của mỗi hàng 
  Values = Rows[i].find_all('td')

  # Lấy ra 7 giá trị tương ứng với 7 cột lấy được lưu vào trong dictionary extracted_data
  if len(Values) == 7:
   RowDict["Date"] = Values[0].find('span').text.replace(',', '')
   RowDict["Open"] = Values[1].find('span').text.replace(',', '')
   RowDict["High"] = Values[2].find('span').text.replace(',', '')
   RowDict["Low"] = Values[3].find('span').text.replace(',', '')
   RowDict["Close"] = Values[4].find('span').text.replace(',', '')
   RowDict["Adj Close"] = Values[5].find('span').text.replace(',', '')
   RowDict["Volume"] = Values[6].find('span').text.replace(',', '')
   extracted_data.append(RowDict)
 except:
  # In ra hàng bị lỗi
  print("Row Number: " + str(i))


    Chuyển dictionary extracted_data thành một Dataframe và lưu lại thành file raw_data.csv

In [12]:
extracted_data = pd.DataFrame(extracted_data)
extracted_data.to_csv('raw data/raw_data.csv',index=False)

,Date,Open,High,Low,Close,Adj Close,Volume
0,May 20 2022,109.57,109.90,105.01,107.59,107.59,99500000
1,May 19 2022,106.28,110.03,106.19,107.32,107.32,88142000
2,May 18 2022,111.44,112.85,106.25,107.11,107.11,108380000
3,May 17 2022,113.28,115.80,111.28,115.37,115.37,76448000
4,May 16 2022,113.10,113.99,110.35,110.81,110.81,74566000
...,...,...,...,...,...,...,...
1759,May 28 2015,21.49,21.57,21.27,21.33,21.33,38248000
1760,May 27 2015,21.37,21.59,21.25,21.57,21.57,44622000
1761,May 26 2015,21.31,21.35,21.10,21.27,21.27,44884000
1762,May 22 2015,21.58,21.62,21.38,21.38,21.38,40412000
